In [75]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
available_pastries = [
    'bolo_arroz',
    'brigadeiro',
    'ovos_moles',
    'pastel_nata',
    'pastel_tentugal']

In [101]:
# Get information from files

width_list = []
height_list = []
X = []
y = []

for filename in os.scandir('raw_data'):
    file_name, file_extension = os.path.splitext(filename.path)
    if filename.is_file() and file_extension == '.jpeg':
        image = Image.open(filename.path)
        X.append(imread(filename.path)[:, :, :1])
        width, height = image.size
        width_list.append(width)
        height_list.append(height)
        for pastry in available_pastries:
            if pastry in file_name:
                y.append(pastry)

X = np.array(X)
y = np.array(y)

print('Minimum height:', min(height_list), height_list.index(min(height_list)))
print('Minimum width:', min(width_list), width_list.index(min(width_list)))

Minimum height: 168 76
Minimum width: 203 98


/var/folders/84/1j64f6yd3yd5g68_pj91ch780000gn/T/ipykernel_40728/1696008496.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [102]:
y = y.reshape(-1, 1)
y

array([['ovos_moles'],
       ['pastel_nata'],
       ['brigadeiro'],
       ['ovos_moles'],
       ['pastel_nata'],
       ['bolo_arroz'],
       ['bolo_arroz'],
       ['pastel_tentugal'],
       ['pastel_tentugal'],
       ['pastel_nata'],
       ['brigadeiro'],
       ['bolo_arroz'],
       ['pastel_nata'],
       ['pastel_tentugal'],
       ['brigadeiro'],
       ['ovos_moles'],
       ['pastel_tentugal'],
       ['pastel_tentugal'],
       ['brigadeiro'],
       ['brigadeiro'],
       ['pastel_tentugal'],
       ['pastel_tentugal'],
       ['ovos_moles'],
       ['pastel_tentugal'],
       ['pastel_nata'],
       ['bolo_arroz'],
       ['brigadeiro'],
       ['pastel_nata'],
       ['pastel_tentugal'],
       ['pastel_tentugal'],
       ['ovos_moles'],
       ['pastel_nata'],
       ['bolo_arroz'],
       ['bolo_arroz'],
       ['brigadeiro'],
       ['pastel_nata'],
       ['ovos_moles'],
       ['ovos_moles'],
       ['bolo_arroz'],
       ['ovos_moles'],
       ['brigadeiro'],

In [103]:
# Encode target y

ohe = OneHotEncoder(sparse = False)
ohe.fit(y)
y_encoded = ohe.transform(y)

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [97]:
def initialize_model():
    
    model = models.Sequential()

    model.add(layers.Conv2D(16, kernel_size=(4, 4), activation='relu', padding='same', input_shape=(76, 78, 1)))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64, kernel_size=(2, 2), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(10, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model  

In [98]:
model = initialize_model()

es = EarlyStopping(patience=10)

history = model.fit(X_train, y_train,
                 epochs=100,
                 batch_size=32, 
                 verbose=1,
                 callbacks=[es],
                 validation_split=0.3)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)